In [1]:
import numpy as np 
import os
import cv2
import mediapipe as mp
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train = (r"G:\Handsign model\this is the last model =))))\training_data")
test = (r"G:\Handsign model\Dataset\asl7gb\Test_Alphabet")

In [3]:
def load_dataset(directory):
    keypoints = []
    labels = []
    for idx, label in enumerate(characters):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            try:
                keypoint = np.load(filepath)
                if not keypoint.size == 0:
                    keypoints.append(keypoint)
                    labels.append(idx)
            except Exception as e:
                print(e)
                print(filepath)
    keypoints = np.array(keypoints)
    labels = np.array(labels)
    return(keypoints, labels)

In [4]:
characters = sorted(os.listdir(train))

In [5]:
characters

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'delete',
 'space']

In [6]:
characters = sorted(os.listdir(train))
keypoints, labels = load_dataset(train)

In [7]:
keypoints.shape

(23392, 16)

In [8]:
new_X = keypoints

In [9]:
new_X.shape

(23392, 16)

In [10]:
new = keypoints * 1.3
new_X = np.concatenate((new_X, new))
labels = np.concatenate((labels, labels))

In [11]:
new = keypoints * 2
new_X = np.concatenate((new_X, new))
labels = np.concatenate((labels, labels))

In [12]:
new = keypoints * 1.7
new_X = np.concatenate((new_X, new))
labels = np.concatenate((labels, labels))

In [13]:
new = keypoints * 0.5 
new_X = np.concatenate((new_X, new))
labels = np.concatenate((labels, labels))

In [14]:
def shuffle_dataset(X, y, random_seed=None):
    # Set a random seed for reproducibility
    np.random.seed(random_seed)
    
    # Get the indices to shuffle the dataset
    indices = np.arange(X.shape[0])
    
    # Shuffle the indices in-place
    np.random.shuffle(indices)
    
    # Shuffle the dataset based on the shuffled indices
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    return X_shuffled, y_shuffled

In [15]:
random_seed = 42  # Set a random seed for reproducibility
X_shuffled, y_shuffled = shuffle_dataset(new_X, labels, random_seed)

In [16]:
test_size = 0.2  # 20% of the data will be used for testing
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=test_size, random_state=random_seed)

In [17]:
def model_eval(model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

### SVM algorithms

In [18]:
from sklearn import svm

In [19]:
svc_clf = svm.SVC(probability=True)

In [20]:
svc_clf.fit(X_train, y_train)

SVC(probability=True)

### Decision tree algorithm

In [19]:
from sklearn import tree

In [20]:
dt_clf = tree.DecisionTreeClassifier()

In [21]:
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

### Random forest algorithm

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rd_clf = RandomForestClassifier(max_depth=5, random_state=0)

In [24]:
rd_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=0)

### Models evaluation

In [21]:
model_eval(svc_clf)

0.9932883036935705

In [22]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [23]:
hands = mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands = 1
) 

In [24]:
def eulidean_distance(landmarkA, landmarkB):
    A = np.array([landmarkA.x, landmarkA.y])
    B = np.array([landmarkB.x, landmarkB.y])
    distance = np.linalg.norm(A-B)
    return distance

In [25]:
def add_distance(landmarks):
    data = []
    data.append(eulidean_distance(landmarks[4], landmarks[0]))
    data.append(eulidean_distance(landmarks[8], landmarks[0]))
    data.append(eulidean_distance(landmarks[12], landmarks[0]))
    data.append(eulidean_distance(landmarks[16], landmarks[0]))
    data.append(eulidean_distance(landmarks[20], landmarks[0]))
    data.append(eulidean_distance(landmarks[4], landmarks[8]))
    data.append(eulidean_distance(landmarks[4], landmarks[12]))
    data.append(eulidean_distance(landmarks[8], landmarks[12]))
    data.append(eulidean_distance(landmarks[12], landmarks[16]))
    data.append(eulidean_distance(landmarks[20], landmarks[16]))
    data.append(eulidean_distance(landmarks[8], landmarks[16]))
    data.append(eulidean_distance(landmarks[8], landmarks[20]))
    data.append(eulidean_distance(landmarks[12], landmarks[20]))
    data.append(eulidean_distance(landmarks[4], landmarks[12]))
    data.append(eulidean_distance(landmarks[4], landmarks[16]))
    data.append(eulidean_distance(landmarks[4], landmarks[20]))
    return data

In [52]:
def process(image, live=True):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    data = []
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark 
        data = add_distance(landmarks)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
    else:
        data = [9 for i in range(0, 16)]
        # Flip the image horizontally for a selfie-view display.
    if live:
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    return data

In [31]:
labels

array([ 0,  0,  0, ..., 23, 23, 23])

In [26]:
import joblib

In [27]:
joblib.dump(svc_clf, 'svm_model.pkl')

['svm_model.pkl']

In [37]:
test = np.load(r"G:\Handsign model\this is the last model =))))\training_data\C\data007.npy")

In [32]:
test.shape

(16,)

In [39]:
pred = svc_clf.predict_proba(test.reshape(1, -1))[0]

In [48]:
np.set_printoptions(suppress=True, precision=6)

In [50]:
pred*100

array([ 0.00027 ,  0.040089, 99.940239,  0.001804,  0.000233,  0.004692,
        0.000147,  0.004211,  0.000089,  0.000527,  0.000934,  0.000256,
        0.000269,  0.000345,  0.000597,  0.001244,  0.000132,  0.000215,
        0.000192,  0.000379,  0.000437,  0.001957,  0.000213,  0.00053 ])

In [41]:
pred[np.argmax(pred)]

0.999402389755485

In [58]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot read video stream")
        break
    data = np.array(process(frame)) 
    if np.all(data == 0):
        print("Not found")
        continue
    classes = svc_clf.predict(data.reshape(1, -1))[0]
    print(characters[classes])
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
